<a href="https://colab.research.google.com/github/gautham-umasankar/Quantum_COVID/blob/master/Classifier_pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Training



##DWave Installation

In [ ]:
!conda --version

/bin/bash: conda: command not found


In [ ]:
!pip install dwave-ocean-sdk


     |████████████████████████████████| 92kB 7.4MB/s 
     |████████████████████████████████| 2.3MB 40.8MB/s 
     |████████████████████████████████| 204kB 49.0MB/s 
     |████████████████████████████████| 92kB 9.0MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 798kB 51.4MB/s 
     |████████████████████████████████| 4.7MB 51.0MB/s 
     |████████████████████████████████| 92kB 7.7MB/s 
     |████████████████████████████████| 399kB 53.5MB/s 
     |████████████████████████████████| 32.7MB 130kB/s 
  Created wheel for pyqubo: filename=pyqubo-0.4.0-cp36-none-any.whl size=40712 sha256=d0e4d256c9aa4ffd2228b7d8d9696645ed006aeb22549faf2461c3d4a03f20b2
  Stored in directory: /root/.cache/pip/wheels/77/b7/47/b989889699f788f4fecdc534bfddbb16df9a9a51f4e431b04d
Successfully built pyqubo


In [ ]:
!dwave setup

Optionally install non-open-source packages and configure your environment.

Do you want to select non-open-source packages to install (y/n)? [y]: y

D-Wave Drivers
These drivers enable some automated performance-tuning features.
This package is available under the D-Wave EULA license.
The terms of the license are available online: https://docs.ocean.dwavesys.com/eula
Install (y/n)? [y]: y
Installing: D-Wave Drivers

D-Wave Problem Inspector
This tool visualizes problems submitted to the quantum computer and the results returned.
This package is available under the D-Wave EULA license.
The terms of the license are available online: https://docs.ocean.dwavesys.com/eula
Install (y/n)? [y]: y
Installing: D-Wave Problem Inspector

Creating the D-Wave configuration file.
Configuration file not found; the default location is: /root/.config/dwave/dwave.conf
Configuration file path [/root/.config/dwave/dwave.conf]: 
Configuration file path does not exist. Create it? [y/N]: y
Profile (create ne

##Imports

In [ ]:
from __future__ import print_function, division
from google.colab import drive
drive.mount('/content/drive', force_remount = True)


!pip install -q git+https://github.com/tensorflow/examples.git

import glob
import pandas  as pd
import numpy   as np
import nibabel as nib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from zipfile import ZipFile
from shutil import copyfile, copyfileobj
import gzip
from tensorflow_examples.models.pix2pix import pix2pix
from IPython.display import clear_output
from PIL import Image
import os
import matplotlib.image as mpimg
from scipy.ndimage import zoom


print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Version:  2.3.0
Eager mode:  True
GPU is NOT AVAILABLE


In [ ]:

import sys
sys.path.append("/content/drive/My Drive/Colab Notebooks/qboost")

from sklearn import preprocessing, metrics
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import load_breast_cancer, fetch_openml
from sklearn.impute import SimpleImputer
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from qboost import WeakClassifiers, QBoostClassifier, QboostPlus

##Classifier function

In [ ]:
def metric(y, y_pred):

    return metrics.accuracy_score(y, y_pred)


In [ ]:
def train_model(X_train, y_train, X_test, y_test, lmd):
    """
    Train qboost model

    :param X_train: train input
    :param y_train: train label
    :param X_test: test input
    :param y_test: test label
    :param lmd: lmbda to control regularization term
    :return:
    """
    NUM_READS = 3000
    NUM_WEAK_CLASSIFIERS = 50
    # lmd = 0.5
    TREE_DEPTH = 3

    # define sampler
    dwave_sampler = DWaveSampler(solver={'qpu': True})
    # sa_sampler = micro.dimod.SimulatedAnnealingSampler()
    emb_sampler = EmbeddingComposite(dwave_sampler)

    N_train = len(X_train)
    N_test = len(X_test)

    print("\n======================================")
    print("Train#: %d, Test: %d" %(N_train, N_test))
    print('Num weak classifiers:', NUM_WEAK_CLASSIFIERS)
    print('Tree depth:', TREE_DEPTH)


    # input: dataset X and labels y (in {+1, -1}

    # Preprocessing data
    # imputer = SimpleImputer()
    scaler = preprocessing.StandardScaler()     # standardize features
    normalizer = preprocessing.Normalizer()     # normalize samples

    # X = imputer.fit_transform(X)
    X_train = scaler.fit_transform(X_train)
    X_train = normalizer.fit_transform(X_train)

    # X_test = imputer.fit_transform(X_test)
    X_test = scaler.fit_transform(X_test)
    X_test = normalizer.fit_transform(X_test)

    ## Adaboost
    print('\nAdaboost')

    clf = AdaBoostClassifier(n_estimators=NUM_WEAK_CLASSIFIERS)

    # scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    print('fitting...')
    clf.fit(X_train, y_train)

    hypotheses_ada = clf.estimators_
    # clf.estimator_weights_ = np.random.uniform(0,1,size=NUM_WEAK_CLASSIFIERS)
    print('testing...')
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)

    print('accu (train): %5.2f'%(metric(y_train, y_train_pred)))
    print('accu (test): %5.2f'%(metric(y_test, y_test_pred)))

    # Ensembles of Decision Tree
    print('\nDecision tree')

    clf2 = WeakClassifiers(n_estimators=NUM_WEAK_CLASSIFIERS, max_depth=TREE_DEPTH)
    clf2.fit(X_train, y_train)

    y_train_pred2 = clf2.predict(X_train)
    y_test_pred2 = clf2.predict(X_test)
    print(clf2.estimator_weights)

    print('accu (train): %5.2f' % (metric(y_train, y_train_pred2)))
    print('accu (test): %5.2f' % (metric(y_test, y_test_pred2)))

    # Ensembles of Decision Tree
    print('\nQBoost')

    DW_PARAMS = {'num_reads': NUM_READS,
                 'auto_scale': True,
                 # "answer_mode": "histogram",
                 'num_spin_reversal_transforms': 10,
                 # 'annealing_time': 10,
                 'postprocess': 'optimization',
                 }

    clf3 = QBoostClassifier(n_estimators=NUM_WEAK_CLASSIFIERS, max_depth=TREE_DEPTH)
    clf3.fit(X_train, y_train, emb_sampler, lmd=lmd, **DW_PARAMS)

    y_train_dw = clf3.predict(X_train)
    y_test_dw = clf3.predict(X_test)

    print(clf3.estimator_weights)

    print('accu (train): %5.2f' % (metric(y_train, y_train_dw)))
    print('accu (test): %5.2f' % (metric(y_test, y_test_dw)))


    # Ensembles of Decision Tree
    print('\nQBoostPlus')
    clf4 = QboostPlus([clf, clf2, clf3])
    clf4.fit(X_train, y_train, emb_sampler, lmd=lmd, **DW_PARAMS)
    y_train4 = clf4.predict(X_train)
    y_test4 = clf4.predict(X_test)
    print(clf4.estimator_weights)

    print('accu (train): %5.2f' % (metric(y_train, y_train4)))
    print('accu (test): %5.2f' % (metric(y_test, y_test4)))


    print("=============================================")
    print("Method \t Adaboost \t DecisionTree \t Qboost \t QboostIt")
    print("Train\t %5.2f \t\t %5.2f \t\t\t %5.2f \t\t %5.2f"% (metric(y_train, y_train_pred),
                                                               metric(y_train, y_train_pred2),
                                                               metric(y_train, y_train_dw),
                                                               metric(y_train, y_train4)))
    print("Test\t %5.2f \t\t %5.2f \t\t\t %5.2f \t\t %5.2f"% (metric(y_test, y_test_pred),
                                                              metric(y_test,y_test_pred2),
                                                              metric(y_test, y_test_dw),
                                                              metric(y_test, y_test4)))
    print("=============================================")

    # plt.subplot(211)
    # plt.bar(range(len(y_test)), y_test)
    # plt.subplot(212)
    # plt.bar(range(len(y_test)), y_test_dw)
    # plt.show()

    return


##Calling the classifier with relevant data

In [ ]:
# #del ncp_scan_list
# del ncp_scan_list
# #del normal_scan_list
# del normal_scan_list
# #del ncp_dataset_list
# #del normal_dataset_list



In [ ]:
n_train = 0.8
n_test = 0.2
path_ncp = '/content/drive/My Drive/Colab Notebooks/COVID19-1/NCP/'
path_cp = '/content/drive/My Drive/Colab Notebooks/CP/'
path_normal = '/content/drive/My Drive/Colab Notebooks/Ordered/'
path_ncp_save = '/content/drive/My Drive/Colab Notebooks/Masks_final/NCP'
path_cp_save = '/content/drive/My Drive/Colab Notebooks/Masks_final/CP'
path_normal_save = '/content/drive/My Drive/Colab Notebooks/Masks_final/Normal'
n_ncp = 30
n_cp = 30
n_normal = 50
masks_ncp = []
masks_cp = []
#masks_normal = []
def mask_reader(path,n):
  mask_list = []
  for i,p in enumerate(os.listdir(path)):
    if i>=n:
      break
    mask = np.load(path + '/' + p)
    mask_list.append(mask)
    #print(mask.shape)
  return mask_list

masks_ncp = mask_reader(path_ncp_save,n_ncp)
masks_cp = mask_reader(path_cp_save,n_cp)
#masks_normal = mask_reader(path_normal_save,n_normal)

patient_list = []

for mask in masks_ncp:
  patient_label = {}
  patient_label['mask'] = mask
  patient_label['label'] = 1
  patient_list.append(patient_label)
  del patient_label

del masks_ncp

for mask in masks_cp:
  patient_label = {}
  patient_label['mask'] = mask
  patient_label['label'] = -1
  patient_list.append(patient_label)
  del patient_label

del masks_cp

# for mask in masks_normal:
#   patient_label = {}
#   patient_label['mask'] = mask
#   patient_label['label'] = -1
#   patient_list.append(patient_label)
#   del patient_label

# del masks_normal

print(len(patient_list))


60


In [ ]:
patient_list[5]['label']

1

In [ ]:
import random
random.shuffle(patient_list)


In [ ]:
for i in patient_list:
  print(i['label'])

-1
1
1
-1
-1
-1
1
1
1
-1
-1
-1
1
-1
-1
1
1
-1
-1
-1
1
-1
-1
1
-1
-1
-1
-1
1
-1
1
-1
-1
1
-1
-1
1
1
1
1
1
1
1
1
-1
1
1
1
-1
-1
1
1
1
1
-1
-1
-1
-1
1
1


In [ ]:
n_total = len(patient_list)
train_data = patient_list[:round(n_train*n_total)]
test_data = patient_list[round(n_train*n_total):]
print(len(train_data),len(test_data))

48 12


In [ ]:
print(type(train_data[0]['mask']),type(train_data[0]['label']))
del patient_list

<class 'numpy.ndarray'> <class 'int'>


In [ ]:
print(train_data[0]['mask'].shape,train_data[1]['mask'].shape)

(224, 224, 90) (224, 224, 90)


In [ ]:
local_vars = list(locals().items())
for var, obj in local_vars:
    print(var, sys.getsizeof(obj))

__name__ 57
__doc__ 113
__package__ 16
__loader__ 16
__spec__ 16
__builtin__ 80
__builtins__ 80
_ih 192
_oh 240
_dh 72
_sh 80
In 192
Out 240
get_ipython 64
exit 56
quit 56
_ 28
__ 53
___ 53
_i 111
_ii 129
_iii 215
_i1 77
_exit_code 24
_i2 61
_i3 960
print_function 56
division 56
drive 80
glob 80
pd 80
np 80
nib 80
plt 80
tf 80
datasets 80
layers 80
models 80
ImageDataGenerator 1056
ZipFile 1056
copyfile 136
copyfileobj 136
gzip 80
pix2pix 80
clear_output 136
Image 80
os 80
mpimg 80
zoom 136
_i4 488
sys 80
preprocessing 80
metrics 80
AdaBoostClassifier 1056
load_breast_cancer 136
fetch_openml 136
SimpleImputer 1056
DWaveSampler 1056
EmbeddingComposite 1056
WeakClassifiers 1056
QBoostClassifier 1056
QboostPlus 1056
_i5 117
metric 136
_i6 4597
train_model 136
_i7 187
_i8 1525
n_train 24
n_test 24
path_ncp 103
path_cp 92
path_normal 97
path_ncp_save 104
path_cp_save 103
path_normal_save 107
n_ncp 28
n_cp 28
n_normal 28
mask_reader 136
mask 36126848
_i9 73
_9 28
_i10 91
random 80
_i11 91
i 

In [ ]:
del mask
# pad = np.zeros((224,224))
# max = 0
# for i in train_data:
#   if i['mask'].shape[2] > max:
#     max = i['mask'].shape[2]
# for i in test_data:
#   if i['mask'].shape[2] > max:
#     max = i['mask'].shape[2]

# print(max)

In [ ]:
# for i in train_data:
#   if i['mask'].shape[2] < max:
#     temp = np.zeros((224,224,max))
#     temp_size = round((max - i['mask'].shape[2])/2.0)
#     temp[:,:,temp_size:(temp_size+i['mask'].shape[2])] = i['mask']
#     i['mask'] = temp

In [ ]:
# for i in test_data:
#   if i['mask'].shape[2] < max:
#     temp = np.zeros((224,224,max))
#     temp_size = round((max - i['mask'].shape[2])/2.0)
#     temp[:,:,temp_size:(temp_size+i['mask'].shape[2])] = i['mask']
#     i['mask'] = temp

In [ ]:
Height = 224
Width = 224
for i in train_data:
  i['mask'] = np.reshape(i['mask'],(Height*Width*90,))
for i in test_data:
  i['mask'] = np.reshape(i['mask'],(Height*Width*90,))


In [ ]:
X_train = np.zeros((round(n_total*n_train),Height*Width*90))
Y_train = np.zeros((round(n_total*n_train),))
X_test = np.zeros((n_total - round(n_total*n_train),Height*Width*90))
Y_test = np.zeros((n_total - round(n_total*n_train),))


for idx,i in enumerate(train_data):
  X_train[idx,:] = i['mask']
  Y_train[idx] = i['label']

for idx,i in enumerate(test_data):
  X_test[idx,:] = i['mask']
  Y_test[idx] = i['label']


In [ ]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
print(Y_train,Y_test)

(48, 4515840) (48,)
(12, 4515840) (12,)
[-1.  1.  1. -1. -1. -1.  1.  1.  1. -1. -1. -1.  1. -1. -1.  1.  1. -1.
 -1. -1.  1. -1. -1.  1. -1. -1. -1. -1.  1. -1.  1. -1. -1.  1. -1. -1.
  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.] [-1. -1.  1.  1.  1.  1. -1. -1. -1. -1.  1.  1.]


In [ ]:
del train_data
del test_data

NameError: ignored

In [ ]:
clfs = train_model(X_train, Y_train, X_test, Y_test, 1.0)



Train#: 48, Test: 12
Num weak classifiers: 50
Tree depth: 3

Adaboost
fitting...
testing...
accu (train):  1.00
accu (test):  0.33

Decision tree
[23.02585093 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093
 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093
 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093
 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093
 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093
 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093
 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093
 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093
 23.02585093 23.02585093]
accu (train):  1.00
accu (test):  0.50

QBoost
[1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 0 1 1 1 1]
accu (train):  1.00
accu (test):  0.67

QBoostPlus
[1 1 1]
accu (train):  1.00
accu (test):  